# Benchmarking feature selection algorithms in the molecular property prediction space: Ames mutagenicity

*Alejandro Corrochano's contribution to the final project.* 

In [1]:
# Datasets
from tdc.single_pred import Tox

# Additional functions in a separate ipynb file
from ipynb.fs.full.AZ_additional_functions import *

# General use 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

#import shap
#Save and load models
import joblib

# RDkit
from rdkit import Chem
from rdkit import RDLogger
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

# Standardizing
from molvs import standardize_smiles

In [2]:
data = Tox(name = 'AMES')
#DF contains the whole dataset stored in a Pandas dataframe format 
df = data.get_data()
#Get_split divides the dataset into 3 different sets (Train, validation, test)
split_AMES = data.get_split()

Found local copy...
Loading...
Done!


In [3]:
data_overview(df, split_AMES)

Total number of molecules: 7278
Train set: 5094 - 70.0%
Validation set: 728 - 10.0%
Test set: 1456 - 20.0%


,Drug_ID,Drug,Y
0,Drug 1,O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2,1
1,Drug 2,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0
2,Drug 3,[N-]=[N+]=CC(=O)NCC(=O)NN,1
3,Drug 4,[N-]=[N+]=C1C=NC(=O)NC1=O,1
4,Drug 6,CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O,1


In [4]:
# Remove fragments of compounds that may contain them
frag_compounds = [comp for comp in df['Drug'] if len(comp.split('.')) > 1]
for ind, c in enumerate(frag_compounds):
    df.loc[df.Drug == frag_compounds[ind], 'Drug'] = frag_compounds[ind].split('.')[0]

print('Number of compounds modified: {}'.format(len(frag_compounds)))

Number of compounds modified: 0


In [5]:
# Duplicates and compounds with less than 5 heavy atoms removal
df = remove_ha_duplicates(df)

Duplicated compounds and with less than 5 heavy atoms have been removed.
New number of compounds: 7140 (-138)


In [6]:
descList = [i for i,j in Descriptors.descList]
#Molecular descriptor calculator
calculator = MolecularDescriptorCalculator(descList)
print('Calculator initialized. Total number of descriptors:', len(descList))

Calculator initialized. Total number of descriptors: 208


In [7]:
# Standardize the molecules 
df['Drug'] = [standardize_smiles(smi) for smi in df['Drug']]

# Generate a list of lists where each row corresponds to a molecule and each column to a descriptor (transpose step required)
c = [calculator.CalcDescriptors(Chem.MolFromSmiles(smi)) for smi in df['Drug']]

# Convert it into a numpy array and transpose it so the columns represent the descriptors
c = np.asarray(c).transpose()

# Append all the descriptors to the DF
for i, descriptor in enumerate(descList):
    df[descriptor] = c[i]
    
#We move the target (Ames) to the end
temp = df.pop('Y') # remove column b and store it in df1
df['Y'] = temp

print('Actual shape of the Dataframe:', df.shape)

KeyboardInterrupt: 

In [ ]:
df.head()

### Generate splits, remove NaNs, and normalization

#### Train, validation, and test sets for both Descriptors and fingerprints

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(df, split_AMES)

#### Remove NaNs values

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = remove_nans(X_train, X_val, X_test, y_train, y_val, y_test)

#### Data normalization required in linear models

In [ ]:
X_train_norm, X_val_norm, X_test_norm = normalize_data(X_train, X_val, X_test)

## Baseline

In [ ]:
models, results = models_comparison(X_train, y_train, X_val, y_val, True, True, False, X_train_norm, X_val_norm)

## MRMR

In [ ]:
models_mrmr, results_mrmr, num_sel_feat_mrmr, features_mrmr = fs_mrmr(X_train, y_train, X_val, y_val, X_train_norm, X_val_norm, True)

In [ ]:
mcc_mrmr, auc_mrmr = plot_fs_metrics(num_sel_feat_mrmr, results_mrmr, 'MRMR', True)

In [ ]:
# Take selected features by MRMR
selected_features_mrmr = features_mrmr[np.argmax(mcc_mrmr)]

In [ ]:
rnd_models_mrmr, metrics_rnd_mrmr = apply_randsearch(X_train[selected_features_mrmr]
                                                        , y_train                                                       
                                                        , X_val[selected_features_mrmr]
                                                        , y_val
                                                        , False
                                                        , X_train_norm[selected_features_mrmr]
                                                        , X_val_norm[selected_features_mrmr])

## RELIEFF

In [ ]:
n_neighbors = 100
models_rf, results_rf, num_sel_feat_rf, features_rf = fs_relieff(X_train, y_train, X_val, y_val, X_train_norm, X_val_norm, True, n_neighbors)

In [ ]:
mcc_rf, auc_rf = plot_classification_metrics(results_rf, X_train, "RelieFf")

In [ ]:
# Take selected features by RelieFf
selected_features_rf = features_rf[np.argmax(mcc_rf)]

In [ ]:
# Hyperparameter optimisation post relief feature selection
rnd_models_rf, metrics_rnd_rf = apply_randsearch(X_train[selected_features_rf]
                                                        , y_train                                                       
                                                        , X_val[selected_features_rf]
                                                        , y_val
                                                        , True
                                                        , X_train_norm[selected_features_rf]
                                                        , X_val_norm[selected_features_rf])

## MIC

In [ ]:
models_mic, results_mic, num_sel_feat_mic, features_mic = fs_score_fn(X_train, y_train, X_val, y_val, X_train_norm, X_val_norm, True, mutual_info_classif)

In [ ]:
mcc_mic, auc_mic = plot_classification_metrics(results_mic, X_train, "Mutual Info Classification")

In [ ]:
selected_features_mic = features_mic[np.argmax(mcc_mic)]

In [ ]:
# Hyperparameter optimisation post mutual info classification feature selection
rnd_models_mic, metrics_rnd_mic = apply_randsearch(X_train[selected_features_mic]
                                                        , y_train                                                       
                                                        , X_val[selected_features_mic]
                                                        , y_val
                                                        , True
                                                        , X_train_norm[selected_features_mic]
                                                        , X_val_norm[selected_features_mic])